In [27]:
import numpy as np,os,sys,glob
import matplotlib.pylab as plt
from ioMicro import *

In [45]:
asm = analysis_smFISH(data_folders = [r'Z:\DCBB_1500gnLib_01_23_2023\MERFISH',r'Z:\DCBB_1500gnLib_01_23_2023\Controls'],
                 save_folder =r'Z:\DCBB_1500gnLib_01_23_2023\MERFISH_Analysis',
                 H0folder=  r'Z:\DCBB_1500gnLib_01_23_2023\Controls\H0*',exclude_H0=False)
#fl_save = r'\\Bbfishdc9\bbfishdc9\DCBBL1_spinal_01_25_2023\psf_Scope2_25_1p5x_cy3_final.pkl'
#dic_psf = pickle.load(open(fl_save,'rb'))
sets = np.unique(['_set'+os.path.basename(fld).split('_set')[-1] for fld in asm.Qfolders if '_set' in os.path.basename(fld)])
print(list(sets))

Found files:731
Found hybe folders:53
['_set1', '_set2']


In [50]:
ifov=250
asm.set_fov(ifov,set_='_set1')
asm.fov =os.path.basename(asm.fl_bk)
MERFISH_flds = [fld for fld in asm.Qfolders if asm.set_ in os.path.basename(fld) if 'MERFISH' in fld]
ctrl_flds  = [fld for fld in asm.Qfolders if asm.set_ in os.path.basename(fld) if ('PTB' in os.path.basename(fld)) or ('H0' in os.path.basename(fld))]
ctrl_flds,MERFISH_flds
ims = [read_im(fld+os.sep+asm.fov) for fld in ctrl_flds+MERFISH_flds]
ncols,sz,sx,sy = ims[0].shape

In [ ]:
im_ref = np.array(ims[0][-1],dtype=np.float32)
drifts = [get_txyz(im[-1],im_ref,sz_norm=20, sz=300) for im in tqdm(ims)]

In [75]:
drift_fl = asm.save_folder+os.sep+'drift_'+asm.fov.split('.')[0]+'.pkl'
pickle.dump([drifts,ctrl_flds+MERFISH_flds,asm.fov],open(drift_fl,'wb'))

In [79]:
def get_iH(fld): return int(os.path.basename(fld).split('_')[0][1:])
inds = np.argsort([get_iH(fld) for fld in ctrl_flds+MERFISH_flds])
all_fld = np.array(ctrl_flds+MERFISH_flds)[inds]
drifts_ = np.array(drifts)[inds]

C:\Users\BintuLab\AppData\Local\Temp\ipykernel_12568\3988061048.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  drifts_ = np.array(drifts)[inds]


In [60]:
ims = [read_im(fld+os.sep+asm.fov) for fld in ctrl_flds+MERFISH_flds]

100%|██████████████████████████████████████████████████████████████████████████████████| 18/18 [07:25<00:00, 24.74s/it]


In [81]:
from dask import array as da
imf = da.concatenate([da.roll(read_im(fld+os.sep+asm.fov),txyz,axis=[1,2,3]).reshape([1,ncols,sz,sx,sy]) 
                      for fld,(txyz,txyzs) in zip(all_fld,drifts_)])

In [87]:
import napari
napari.view_image(imf)

Viewer(axes=Axes(visible=False, labels=True, colored=True, dashed=False, arrows=True), camera=Camera(center=(0.0, 1499.5, 1499.5), zoom=0.37461666666666665, angles=(0.0, 0.0, 90.0), perspective=0.0, interactive=True), cursor=Cursor(position=(1.0, 1.0, 0.0, 0.0, 0.0), scaled=True, size=1, style=<CursorStyle.STANDARD: 'standard'>), dims=Dims(ndim=5, ndisplay=2, last_used=0, range=((0.0, 18.0, 1.0), (0.0, 4.0, 1.0), (0.0, 37.0, 1.0), (0.0, 3000.0, 1.0), (0.0, 3000.0, 1.0)), current_step=(9, 2, 18, 1500, 1500), order=(0, 1, 2, 3, 4), axis_labels=('0', '1', '2', '3', '4')), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False), layers=[<Image layer 'imf' at 0x13f7c9f9280>], scale_bar=ScaleBar(visible=False, colored=False, ticks=True, position=<Position.BOTTOM_RIGHT: 'bottom_right'>, font_size=10, unit=None), text_overlay=TextOverlay(visible=False, color=(0.5, 0.5, 0.5, 1.0), font_size=10, position=<TextOverlayPosition.TOP_LEFT: 'top_left'>, text=''), overlays=Overlays(interaction_box=Int

In [86]:
import pandas
codebook = pandas.read_csv(r'Z:\DCBB_1500gnLib_01_23_2023\MERFISH\New_DCBB-300_MERFISH_encoding_2_21_2023.csv')
import numpy as np
code_gene = codebook[codebook['name']=='Ptbp1'].iloc[:,2:]
code = np.where(code_gene.iloc[0])[0]
icol =code[0]%3 
hybes =code//3
hybes+2,icol

(array([2, 3, 6, 9], dtype=int64), 2)

In [89]:
import pandas
codebook = pandas.read_csv(r'Z:\DCBB_1500gnLib_01_23_2023\MERFISH\New_DCBB-300_MERFISH_encoding_2_21_2023.csv')
import numpy as np
code_gene = codebook[codebook['name']=='Igfbpl1'].iloc[:,2:]
code = np.where(code_gene.iloc[0])[0]
icol =code[0]%3 
hybes =code//3 
hybes+2,icol

(array([2, 5, 7, 8], dtype=int64), 0)

In [20]:
set_flds = [fld for fld in asm.Qfolders if asm.set_ in os.path.basename(fld)]
iPTB = [ifld for ifld,fld in enumerate(set_flds) if 'PTB' in os.path.basename(fld)][0]
iPTB

21

In [21]:
asm.set_hybe(iPTB)

completed = asm.check_finished_file()
if not completed:
    asm.get_background(force=False)
    asm.get_signal()
    asm.compute_drift(sz=300)
    #asm.get_aligned_ims()
    #imf = asm.im_sig__/asm.im_bk__
    #resc=5
    #asm.im_ration = np.array([norm_slice(im_,s=30)/np.median(np.abs(im_[::resc,::resc,::resc])) for im_ in asm.im_sig_])
    #asm.im_ration = np.array([norm_slice(im_,s=30)/np.median(np.abs(im_[::resc,::resc,::resc])) for im_ in asm.im_sig_])

    asm.dic_th = {0:1.25,1:0.5,2:1.25}
    asm.get_Xh(th = 3)
    asm.save_fits(plt_val=False)

### define H0 and load background
Background file: Z:\DCBB_1500gnLib_01_23_2023\Controls\H0_set1\Conv_zscan__250.zarr
### load signal
Signal file: Z:\DCBB_1500gnLib_01_23_2023\Controls\H1_PTB_set1\Conv_zscan__250.zarr
Found drift:[ 7 -2  5]


NameError: name 'dic_psf' is not defined

In [26]:
import napari
napari.view_image(asm.im_sig_)

Viewer(axes=Axes(visible=False, labels=True, colored=True, dashed=False, arrows=True), camera=Camera(center=(0.0, 1499.5, 1499.5), zoom=0.38158333333333333, angles=(0.0, 0.0, 90.0), perspective=0.0, interactive=True), cursor=Cursor(position=(1.0, 1.0, 0.0, 0.0), scaled=True, size=1, style=<CursorStyle.STANDARD: 'standard'>), dims=Dims(ndim=4, ndisplay=2, last_used=0, range=((0.0, 4.0, 1.0), (0.0, 37.0, 1.0), (0.0, 3000.0, 1.0), (0.0, 3000.0, 1.0)), current_step=(2, 18, 1500, 1500), order=(0, 1, 2, 3), axis_labels=('0', '1', '2', '3')), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False), layers=[<Image layer 'Image' at 0x13cf6fd7ac0>], scale_bar=ScaleBar(visible=False, colored=False, ticks=True, position=<Position.BOTTOM_RIGHT: 'bottom_right'>, font_size=10, unit=None), text_overlay=TextOverlay(visible=False, color=(0.5, 0.5, 0.5, 1.0), font_size=10, position=<TextOverlayPosition.TOP_LEFT: 'top_left'>, text=''), overlays=Overlays(interaction_box=InteractionBox(points=None, show=Fa

In [22]:
?asm.get_Xh

In [16]:
#asm.Qfolders = [asm.Qfolders]


['Z:\\DCBB_1500gnLib_01_23_2023\\MERFISH\\H10_MERRep_set1',
 'Z:\\DCBB_1500gnLib_01_23_2023\\MERFISH\\H10_MERRep_set2',
 'Z:\\DCBB_1500gnLib_01_23_2023\\MERFISH\\H11_MERRep_set1',
 'Z:\\DCBB_1500gnLib_01_23_2023\\MERFISH\\H11_MERRep_set2',
 'Z:\\DCBB_1500gnLib_01_23_2023\\MERFISH\\H12_MERRep_set1',
 'Z:\\DCBB_1500gnLib_01_23_2023\\MERFISH\\H12_MERRep_set2',
 'Z:\\DCBB_1500gnLib_01_23_2023\\MERFISH\\H13_MERRep_set1',
 'Z:\\DCBB_1500gnLib_01_23_2023\\MERFISH\\H13_MERRep_set2',
 'Z:\\DCBB_1500gnLib_01_23_2023\\MERFISH\\H14_MER_set1',
 'Z:\\DCBB_1500gnLib_01_23_2023\\MERFISH\\H14_MER_set2',
 'Z:\\DCBB_1500gnLib_01_23_2023\\MERFISH\\H15_MER_set1',
 'Z:\\DCBB_1500gnLib_01_23_2023\\MERFISH\\H15_MER_set2',
 'Z:\\DCBB_1500gnLib_01_23_2023\\MERFISH\\H16_MER_set1',
 'Z:\\DCBB_1500gnLib_01_23_2023\\MERFISH\\H16_MER_set2',
 'Z:\\DCBB_1500gnLib_01_23_2023\\MERFISH\\H1_MERRep_set1',
 'Z:\\DCBB_1500gnLib_01_23_2023\\MERFISH\\H1_MERRep_set2',
 'Z:\\DCBB_1500gnLib_01_23_2023\\MERFISH\\H2_MER_set1',
 'Z:

In [8]:
?asm.set_fov

In [ ]:
def process_fov(ifov):
    asm.set_fov(ifov)
    final_segmentation(asm.fl_bk,
                        analysis_folder=asm.save_folder,
                        plt_val=True,
                        rescz = 3,trimz=2, resc=4,p99=850)
    if True:
        for iQ in range(nQs):

            asm.set_hybe(iQ)

            completed = asm.check_finished_file()
            if not completed:
                asm.get_background(force=False)
                asm.get_signal()
                asm.compute_drift(sz=300)
                #asm.get_aligned_ims()
                #imf = asm.im_sig__/asm.im_bk__
                resc=5
                #asm.im_ration = np.array([norm_slice(im_,s=30)/np.median(np.abs(im_[::resc,::resc,::resc])) for im_ in asm.im_sig_])
                #asm.im_ration = np.array([norm_slice(im_,s=30)/np.median(np.abs(im_[::resc,::resc,::resc])) for im_ in asm.im_sig_])

                asm.dic_th = {0:1.25,1:0.5,2:1.25}
                asm.get_Xh(th = 3,dic_psf=dic_psf)
                asm.save_fits(plt_val=False)